In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls /content/drive/My\ Drive/Colab\ Notebooks

 558_Assignments_Code  'Fondation Final Project'   my_own_code
 data		        Kaggle			  'powerful word.ipynb'


In [3]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import pandas as pd
from gensim.models import KeyedVectors
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Similarity Matrix

In [20]:
similarity_matrix = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/medical data/similarity_matrix_2.csv')
similarity_matrix

,Unnamed: 0,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42,A43,A44,A46,...,Y50,Y51,Y52,Y53,Y54,Y55,Y56,Y57,Y58,Y59,Y60,Y61,Y62,Y63,Y64,Y65,Y66,Y69,Y70,Y71,Y72,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89,max_index,max_prob
0,0,0.026074,0.448598,0.215652,0.169547,0.085681,-0.034165,0.205466,0.203477,0.360185,0.453587,0.077050,0.182978,0.377504,0.252529,0.128231,0.369948,0.127808,0.098059,0.252529,0.433509,0.131187,0.130110,0.274488,0.203477,0.205466,0.127808,0.178459,0.178459,0.178459,0.142052,0.136957,0.448598,0.252529,-0.028994,0.050428,0.306242,0.127808,0.127808,0.451563,...,0.186743,0.209723,0.091409,0.360302,0.123843,0.054124,0.331416,0.030469,0.099248,0.058531,-0.040710,-0.040710,-0.040710,0.199004,0.220156,0.027234,0.162231,0.153140,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.130959,0.199996,0.173952,-0.040710,0.271380,0.294085,0.064202,0.173952,0.180391,J30,0.754661
1,1,0.019456,0.151921,0.013340,-0.012869,0.039627,-0.002846,0.142382,0.147403,0.049392,0.239336,0.068022,0.072123,0.060074,0.128047,0.026142,-0.075832,0.148826,-0.022669,0.128047,0.123034,-0.062104,0.004337,-0.005808,0.147403,0.142382,0.148826,0.049344,0.049344,0.049344,-0.049913,0.064610,0.151921,0.128047,0.039291,-0.033113,0.044011,0.148826,0.148826,0.012578,...,-0.052672,0.060649,0.008961,0.231358,-0.059652,0.122590,-0.004250,-0.011132,0.078985,0.017546,0.122950,0.122950,0.122950,0.164966,0.061099,0.059107,0.090576,0.236482,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.228241,0.176164,0.202111,0.122950,0.102914,0.079242,0.195165,0.202111,0.133385,D50,0.548074
2,2,-0.014939,0.163079,0.045342,0.044433,0.017048,0.016677,0.150356,0.241952,0.137304,0.224940,0.143417,0.077072,0.070516,0.268778,0.123623,-0.026696,0.041743,-0.044580,0.268778,0.123922,-0.076073,0.088760,0.049214,0.241952,0.150356,0.041743,0.039990,0.039990,0.039990,-0.048133,0.000101,0.163079,0.268778,0.036089,-0.034964,0.177737,0.041743,0.041743,0.092559,...,-0.007619,0.145603,0.006860,0.179582,-0.006890,0.101605,-0.009698,-0.037727,-0.000692,-0.011931,0.215078,0.215078,0.215078,0.175787,0.038003,0.070912,0.117934,0.230076,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.247539,0.202966,0.232664,0.215078,0.194271,0.170870,0.248808,0.232664,0.172687,H59,0.662815
3,3,0.026513,0.135662,0.039851,0.003296,0.032318,0.082930,0.101269,0.167935,0.175855,0.137657,0.110944,0.091970,0.083633,0.184592,0.095259,0.021598,0.042892,0.079491,0.184592,0.069730,-0.029300,-0.007063,0.060740,0.167935,0.101269,0.042892,0.021652,0.021652,0.021652,0.087548,-0.028894,0.135662,0.184592,0.001706,0.012830,0.172901,0.042892,0.042892,0.041767,...,-0.047537,0.024533,0.043092,0.030223,0.058737,0.127166,-0.029659,0.018877,-0.006526,0.078879,0.192056,0.192056,0.192056,0.080938,-0.034227,0.227200,0.096190,0.004380,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.112916,0.072081,0.091467,0.192056,0.101694,0.046089,0.081032,0.091467,-0.001974,I50,0.671866
4,4,0.011046,0.108014,-0.001801,-0.040168,0.008490,0.026372,0.095513,0.152703,0.130312,0.108499,0.141786,0.066100,0.145210,0.169432,0.047066,-0.015027,0.064869,0.032353,0.169432,0.032554,-0.051156,0.004553,0.042336,0.152703,0.095513,0.064869,0.008141,0.008141,0.008141,0.026661,-0.024269,0.108014,0.169432,-0.005614,-0.037363,0.221342,0.064869,0.064869,0.052019,...,-0.038899,0.085043,0.061453,0.064518,0.038458,0.201387,0.010800,0.040443,-0.005056,0.015204,0.167040,0.167040,0.167040,0.088708,0.001563,0.256101,0.064527,0.021454,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.092948,0.113825,0.097463,0.167040,0.134355,0.035131,0.085959,0.097463,0.004841,I50,0.683359
...,...,...,...,...,...,...,...,...,...,

In [22]:
similarity_matrix['max_prob'].describe()

count    4999.000000
mean        0.661345
std         0.112378
min         0.292794
25%         0.586751
50%         0.664255
75%         0.741693
max         0.983962
Name: max_prob, dtype: float64

In [0]:
overthreshold_ones = similarity_matrix[similarity_matrix['max_prob']>0.75]

In [24]:
overthreshold_ones[['max_index','max_prob']]

,max_index,max_prob
0,J30,0.754661
8,C76,0.864232
9,I50,0.759305
13,H59,0.803992
14,S02,0.760790
...,...,...
4981,I82,0.831553
4982,H59,0.753036
4991,W76,0.756710
4992,W76,0.849206


In [25]:
from sklearn.utils import shuffle
shuffle(overthreshold_ones)[:10]['max_index']

3450    P02
3225    R71
2072    T06
2292    S88
2198    S88
3612    K51
3990    E66
1222    K36
957     N83
1275    T06
Name: max_index, dtype: object

In [26]:
shuffle(overthreshold_ones)[:10]['max_index']

2460    R50
822     T16
3026    C76
1850    S88
4878    R00
295     H92
2069    S42
3297    R07
4299    D24
2266    R07
Name: max_index, dtype: object

In [0]:
mt[2215:2216]['description'].values

In [28]:
similarity_matrix['over_threshold'] = ''
similarity_matrix['over_threshold'].iloc[overthreshold_ones.index] = overthreshold_ones['max_index']

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [29]:
similarity_matrix

,Unnamed: 0,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42,A43,A44,A46,...,Y51,Y52,Y53,Y54,Y55,Y56,Y57,Y58,Y59,Y60,Y61,Y62,Y63,Y64,Y65,Y66,Y69,Y70,Y71,Y72,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89,max_index,max_prob,over_threshold
0,0,0.026074,0.448598,0.215652,0.169547,0.085681,-0.034165,0.205466,0.203477,0.360185,0.453587,0.077050,0.182978,0.377504,0.252529,0.128231,0.369948,0.127808,0.098059,0.252529,0.433509,0.131187,0.130110,0.274488,0.203477,0.205466,0.127808,0.178459,0.178459,0.178459,0.142052,0.136957,0.448598,0.252529,-0.028994,0.050428,0.306242,0.127808,0.127808,0.451563,...,0.209723,0.091409,0.360302,0.123843,0.054124,0.331416,0.030469,0.099248,0.058531,-0.040710,-0.040710,-0.040710,0.199004,0.220156,0.027234,0.162231,0.153140,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.199996,0.130959,0.199996,0.173952,-0.040710,0.271380,0.294085,0.064202,0.173952,0.180391,J30,0.754661,J30
1,1,0.019456,0.151921,0.013340,-0.012869,0.039627,-0.002846,0.142382,0.147403,0.049392,0.239336,0.068022,0.072123,0.060074,0.128047,0.026142,-0.075832,0.148826,-0.022669,0.128047,0.123034,-0.062104,0.004337,-0.005808,0.147403,0.142382,0.148826,0.049344,0.049344,0.049344,-0.049913,0.064610,0.151921,0.128047,0.039291,-0.033113,0.044011,0.148826,0.148826,0.012578,...,0.060649,0.008961,0.231358,-0.059652,0.122590,-0.004250,-0.011132,0.078985,0.017546,0.122950,0.122950,0.122950,0.164966,0.061099,0.059107,0.090576,0.236482,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.176164,0.228241,0.176164,0.202111,0.122950,0.102914,0.079242,0.195165,0.202111,0.133385,D50,0.548074,
2,2,-0.014939,0.163079,0.045342,0.044433,0.017048,0.016677,0.150356,0.241952,0.137304,0.224940,0.143417,0.077072,0.070516,0.268778,0.123623,-0.026696,0.041743,-0.044580,0.268778,0.123922,-0.076073,0.088760,0.049214,0.241952,0.150356,0.041743,0.039990,0.039990,0.039990,-0.048133,0.000101,0.163079,0.268778,0.036089,-0.034964,0.177737,0.041743,0.041743,0.092559,...,0.145603,0.006860,0.179582,-0.006890,0.101605,-0.009698,-0.037727,-0.000692,-0.011931,0.215078,0.215078,0.215078,0.175787,0.038003,0.070912,0.117934,0.230076,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.202966,0.247539,0.202966,0.232664,0.215078,0.194271,0.170870,0.248808,0.232664,0.172687,H59,0.662815,
3,3,0.026513,0.135662,0.039851,0.003296,0.032318,0.082930,0.101269,0.167935,0.175855,0.137657,0.110944,0.091970,0.083633,0.184592,0.095259,0.021598,0.042892,0.079491,0.184592,0.069730,-0.029300,-0.007063,0.060740,0.167935,0.101269,0.042892,0.021652,0.021652,0.021652,0.087548,-0.028894,0.135662,0.184592,0.001706,0.012830,0.172901,0.042892,0.042892,0.041767,...,0.024533,0.043092,0.030223,0.058737,0.127166,-0.029659,0.018877,-0.006526,0.078879,0.192056,0.192056,0.192056,0.080938,-0.034227,0.227200,0.096190,0.004380,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.072081,0.112916,0.072081,0.091467,0.192056,0.101694,0.046089,0.081032,0.091467,-0.001974,I50,0.671866,
4,4,0.011046,0.108014,-0.001801,-0.040168,0.008490,0.026372,0.095513,0.152703,0.130312,0.108499,0.141786,0.066100,0.145210,0.169432,0.047066,-0.015027,0.064869,0.032353,0.169432,0.032554,-0.051156,0.004553,0.042336,0.152703,0.095513,0.064869,0.008141,0.008141,0.008141,0.026661,-0.024269,0.108014,0.169432,-0.005614,-0.037363,0.221342,0.064869,0.064869,0.052019,...,0.085043,0.061453,0.064518,0.038458,0.201387,0.010800,0.040443,-0.005056,0.015204,0.167040,0.167040,0.167040,0.088708,0.001563,0.256101,0.064527,0.021454,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.113825,0.092948,0.113825,0.097463,0.167040,0.134355,0.035131,0.085959,0.097463,0.004841,I50,0.683359,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [0]:
icdto27 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/map data/ICDto27.csv')
icdto27
dic = dict()
for i in range(len(icdto27)):
  dic[icdto27['ICD'][i]] = icdto27['27'][i]
similarity_matrix['Rankable_Cause'] = similarity_matrix['over_threshold'].map(dic)

In [0]:
similarity_matrix['Rankable_Cause'] = similarity_matrix['Rankable_Cause'].fillna(99)

In [0]:
similarity_matrix['Rankable_Cause'] = similarity_matrix['Rankable_Cause'].astype(int)

In [0]:
n,v = np.unique(similarity_matrix['Rankable_Cause'], return_counts=True)

In [36]:
similarity_matrix[similarity_matrix['Rankable_Cause'].isin([7,4,13,9])]

,Unnamed: 0,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42,A43,A44,A46,...,Y52,Y53,Y54,Y55,Y56,Y57,Y58,Y59,Y60,Y61,Y62,Y63,Y64,Y65,Y66,Y69,Y70,Y71,Y72,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89,max_index,max_prob,over_threshold,Rankable_Cause
8,8,0.001558,0.370480,0.110712,0.046477,-0.011164,0.061519,0.116865,-0.003429,0.135725,0.325740,0.232688,0.060054,0.501889,0.126150,0.152845,0.111613,-0.030805,0.001192,0.126150,0.324436,0.049675,0.064617,0.109798,-0.003429,0.116865,-0.030805,0.096706,0.096706,0.096706,0.036757,-0.043599,0.370480,0.126150,-0.034266,-0.044785,0.368585,-0.030805,-0.030805,0.392384,...,-0.058430,0.207403,0.064205,0.457653,0.163334,-0.001829,-0.012280,-0.032414,0.205767,0.205767,0.205767,0.009893,-0.082362,0.167399,0.097245,0.107434,0.126034,0.126034,0.126034,0.126034,0.126034,0.126034,0.126034,0.126034,0.126034,0.126034,0.126034,0.098263,0.126034,0.057863,0.205767,0.169435,0.137118,0.059048,0.057863,0.154517,C76,0.864232,C76,4
9,9,-0.005937,0.020316,-0.050340,-0.044052,-0.026607,0.072956,0.038661,0.103758,0.094686,-0.002387,0.053660,-0.021733,-0.003035,0.091453,-0.018576,-0.083469,0.075121,0.045501,0.091453,0.000415,-0.000646,-0.045896,-0.076120,0.103758,0.038661,0.075121,-0.007868,-0.007868,-0.007868,0.034593,0.020090,0.020316,0.091453,-0.015496,-0.050033,-0.047783,0.075121,0.075121,-0.126025,...,0.010836,-0.011005,0.064579,0.140405,-0.081826,0.047353,0.058838,0.060283,0.207659,0.207659,0.207659,0.059616,-0.031942,0.136947,0.041887,-0.009642,0.066383,0.066383,0.066383,0.066383,0.066383,0.066383,0.066383,0.066383,0.066383,0.066383,0.066383,0.067391,0.066383,0.057670,0.207659,0.059855,0.027475,0.037217,0.057670,-0.046296,I50,0.759305,I50,7
26,26,0.045209,0.264885,0.096483,0.105378,0.048931,0.030070,0.165845,0.176200,0.233222,0.298353,0.286533,0.063063,0.314064,0.195812,0.112357,0.077628,0.044382,0.075596,0.195812,0.202218,-0.028547,0.078165,0.091061,0.176200,0.165845,0.044382,0.094610,0.094610,0.094610,0.050370,-0.022775,0.264885,0.195812,-0.015422,0.011561,0.334660,0.044382,0.044382,0.224205,...,0.032144,0.271031,0.097420,0.260639,0.143067,-0.006558,-0.021588,0.029727,0.315665,0.315665,0.315665,0.191452,0.049148,0.221183,0.055180,0.114589,0.187238,0.187238,0.187238,0.187238,0.187238,0.187238,0.187238,0.187238,0.187238,0.187238,0.187238,0.153923,0.187238,0.195430,0.315665,0.212236,0.169691,0.313344,0.195430,0.139079,C67,0.792700,C67,4
32,32,0.058623,0.183962,0.077898,0.070585,0.062846,-0.018388,0.093555,0.192002,0.166732,0.233999,0.172495,0.071362,0.231096,0.154547,0.109667,0.072465,0.044799,0.044663,0.154547,0.154407,-0.008963,0.113368,0.085209,0.192002,0.093555,0.044799,0.014690,0.014690,0.014690,-0.013648,-0.006761,0.183962,0.154547,-0.019754,0.004731,0.421167,0.044799,0.044799,0.214936,...,0.009825,0.167436,0.072184,0.217671,0.131885,0.066150,-0.032870,0.041596,0.334263,0.334263,0.334263,0.054397,-0.008340,0.207090,0.017655,0.139095,0.178307,0.178307,0.178307,0.178307,0.178307,0.178307,0.178307,0.178307,0.178307,0.178307,0.178307,0.073701,0.178307,0.138557,0.334263,0.161317,0.069880,0.114412,0.138557,0.081409,C66,0.766647,C66,4
38,38,-0.014207,0.095174,0.066883,0.107322,0.088852,-0.011335,0.175271,0.221511,0.142833,0.118130,0.346685,0.106074,0.264465,0.067670,0.057872,0.054945,0.039923,0.049068,0.067670,0.058626,-0.060142,0.009459,0.061328,0.221511,0.175271,0.039923,0.072664,0.072664,0.072664,0.068056,-0.028676,0.095174,0.067670,-0.064768,-0.001327,0.314390,0.039923,0.039923,0.121824,...,0.039925,0.119318,0.110326,0.179724,0.068165,-0.006684,0.021658,0.056240,0.340831,0.340831,0.340831,0.101711,0.013286,0.149303,-0.033474,0.054993,0.084945,0.084945,0.084945,0.084945,0.084945,0.084945,0.084945,0.084945,0.084945,0.084945,0.084945,0.081791,0.084945,0.080192,0.340831,0.121695,0.039596,0.373981,0.080192,0.102569,C67,0.797874,C67,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [0]:
topcause_ones = similarity_matrix[similarity_matrix['Rankable_Cause'].isin([7,4,13,9])]['Rankable_Cause']
topcause_ones = topcause_ones.to_frame()

In [48]:
topcause_ones.index.rename('Medical Case #', inplace=True)
topcause_ones

,Rankable_Cause
Medical Case #,
8,4
9,7
26,4
32,4
38,4
...,...
4908,7
4917,9
4952,9


In [0]:
topcause_ones.to_csv('/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/medical data/topcause_ones.csv')

In [0]:
similarity_matrix.to_csv('/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/medical data/similarity_matrix_3.csv')